# Queries with and without Azure OpenAI

이전 단계들에서는 서로 다른 두 데이터 소스에서 검색 엔진을 로드했습니다. 이번에는 몇 가지 예제 쿼리를 시도한 다음 Azure OpenAI 서비스를 사용하여 더 나은 결과를 얻을 수 있는지 알아보겠습니다.

이 **Multi-Index** 데모는 회사가 서로 다른 유형의 문서와 완전히 다른 주제를 로드하고 검색 엔진이 가장 연관성이 높은 결과로 응답해야 하는 시나리오를 모방합니다.

## Set up variables

In [1]:
import os
import urllib
import requests
import random
import json
from collections import OrderedDict
from IPython.display import display, HTML, Markdown
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.llms import AzureOpenAI
from langchain.chat_models import AzureChatOpenAI
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document
from langchain.chains.question_answering import load_qa_chain
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.embeddings import OpenAIEmbeddings

from common.prompts import COMBINE_QUESTION_PROMPT, COMBINE_PROMPT, COMBINE_PROMPT_TEMPLATE
from common.utils import (
    get_search_results,
    model_tokens_limit,
    num_tokens_from_docs,
    num_tokens_from_string
)

from dotenv import load_dotenv
load_dotenv("credentials.env")

True

In [2]:
# Setup the Payloads header
headers = {'Content-Type': 'application/json','api-key': os.environ['AZURE_SEARCH_KEY']}
params = {'api-version': os.environ['AZURE_SEARCH_API_VERSION']}

## 멀티 검색 인덱스 쿼리

In [3]:
# 조회할 텍스트 기반 인덱스
index1_name = "cogsrch-index-files"
indexes = [index1_name]
# index2_name = "cogsrch-index-csv"
# indexes = [index1_name, index2_name]

In [4]:
QUESTION = "블록체인의 해시함수가 뭐야?"
# QUESTION = "발달상권 가구의 4분기 분기매출을 알려줘"

### 두 인덱스를 개별적으로 검색하고 결과를 집계합니다

#### **Note**: 
인덱스들을 표준화하기 위해 각 텍스트 기반 인덱스에 **'id, title, content, chunk, language, name, location, vectorized'** 라는 8개의 필수 필드가 존재해야 합니다. 이는 코드를 따라 각 문서가 동일하게 취급될 수 있도록 하기 위한 것입니다. 또한 모든 인덱스는 의미 구성을 가져야 합니다.

In [5]:
agg_search_results = dict()

for index in indexes:
    search_payload = {
        "search": QUESTION,
        "select": "id, title, chunks, language, name, location",
        "queryType": "semantic",
        "semanticConfiguration": "my-semantic-config",
        "count": "true",
        "speller": "lexicon",
        "queryLanguage": "en-us",
        "captions": "extractive",
        "answers": "extractive",
        "top": "10"
    }

    r = requests.post(os.environ['AZURE_SEARCH_ENDPOINT'] + "/indexes/" + index + "/docs/search",
                     data=json.dumps(search_payload), headers=headers, params=params)
    print(r.status_code)

    search_results = r.json()
    agg_search_results[index]=search_results
    print("Index:", index, "Results Found: {}, Results Returned: {}".format(search_results['@odata.count'], len(search_results['value'])))

200
Index: cogsrch-index-files Results Found: 1, Results Returned: 1


### Search Score를 기준으로 상위 결과 표시

In [6]:
agg_search_results[index]

{'@odata.context': "https://cog-search-iwdkwba7nutto.search.windows.net/indexes('cogsrch-index-files')/$metadata#docs(*)",
 '@odata.count': 1,
 '@search.answers': [{'key': 'aHR0cHM6Ly9ob2xzdG9yYWdlLmJsb2IuY29yZS53aW5kb3dzLm5ldC9hcnhpdmNzL2Jsb2NrY2hhaW5fZXhwbGFpbmF0aW9uLnBkZg2',
   'text': '블록체인에서는 해시함수를 사용하는 3가지 목적을 살펴보면 첫째, 공개키의 해시  값을 지갑주소로 활용하여 익명화된 거래를 수행하고, 가상 화폐의 전자지갑 주소는   공개키 기반 암호화 알고리즘에서 생성된 공개키의 해시값을 사용한다. 개인정보(정확  히는 송신자의 계좌정보) 없이 익명화된 거래를 통해 송금자의 신원을 감추고, 송금 할   수 있다. 둘째, 해시함수를 사용하여 2가지의 무결성 검증에 사용하게 된다. 체인으로   연결된 블록헤더의 해시값을 활용하여, 해시값 체인으로 연결된 블록의 무결성 검증에   사용된다.',
   'highlights': '블록체인에서는 해시함수를 사용하는 3가지 목적을 살펴보면 첫째, 공개키의 해시  값을 지갑주소로 <em>활용하여 익명화된 거래를 수행하고,</em> 가상 화폐의 전자지갑 주소는   공개키 기반 암호화 알고리즘에서 생성된 공개키의 해시값을 사용한다. 개인정보(정확  히는 송신자의 계좌정보) 없이 익명화된 거래를 통해 송금자의 신원을 감추고, 송금 할   수 있다. 둘째, 해시함수를 사용하여 2가지의<em> 무결성 검증에</em> 사용하게 된다. 체인으로   연결된 블록헤더의 해시값을 활용하여, 해시값 체인으로 연결된 블록의 무결성 검증에   사용된다.',
   'score': 0.8818359375}],
 'value': [{'@search.score': 0.87794375,
   '@search.re

In [7]:
display(HTML('<h4>Top Answers</h4>'))

for index,search_results in agg_search_results.items():
    for result in search_results['@search.answers']:
        if result['score'] > 0.5: # Show answers that are at least 50% of the max possible score=1
            display(HTML('<h5>' + 'Answer - score: ' + str(round(result['score'],2)) + '</h5>'))
            display(HTML(result['text']))
            
print("\n\n")
display(HTML('<h4>Top Results</h4>'))

content = dict()
ordered_content = OrderedDict()


for index,search_results in agg_search_results.items():
    for result in search_results['value']:
        if result['@search.rerankerScore'] > 1:# Show answers that are at least 25% of the max possible score=4
            content[result['id']]={
                                    "title": result['title'],
                                    "chunks": result['chunks'],
                                    "language": result['language'], 
                                    "name": result['name'], 
                                    "location": result['location'] ,
                                    "caption": result['@search.captions'][0]['text'],
                                    "score": result['@search.rerankerScore'],
                                    "index": index
                                    }
    
#After results have been filtered we will Sort and add them as an Ordered list\n",
for id in sorted(content, key= lambda x: content[x]["score"], reverse=True):
    ordered_content[id] = content[id]
    url = str(ordered_content[id]['location']) + os.environ['BLOB_SAS_TOKEN']
    title = str(ordered_content[id]['title']) if (ordered_content[id]['title']) else ordered_content[id]['name']
    score = str(round(ordered_content[id]['score'],2))
    display(HTML('<h5><a href="'+ url + '">' + title + '</a> - score: '+ score + '</h5>'))
    display(HTML(ordered_content[id]['caption']))

# Azure OpenAI 사용

검색 결과에서 GPT 모델까지의 답변과 문서 내용을 컨텍스트로 제공하고 더 나은 응답을 제공하기 까지 몇가지 작업을 이해해야 합니다. 

1) 체인닝 및 신속 엔지니어링
2) 임베딩

In [8]:
# Set the ENV variables that Langchain needs to connect to Azure OpenAI
os.environ["OPENAI_API_BASE"] = os.environ["AZURE_OPENAI_ENDPOINT"]
os.environ["OPENAI_API_KEY"] = os.environ["AZURE_OPENAI_API_KEY"]
os.environ["OPENAI_API_VERSION"] = os.environ["AZURE_OPENAI_API_VERSION"]
os.environ["OPENAI_API_TYPE"] = "azure"

**Important Note**
이번 단계부터는 OpenAI 모델을 사용할 예정입니다. Azure OpenAI 포털 내에 아래 모델을 배포했는지 확인하십시오:

- text-embedding-ada-002
- gpt-35-turbo
- gpt-35-turbo-16k
- gpt-4
- gpt-4-32k

만약 위에 주어진 이름 말고 다른 이름으로 모델을 배포했다면 아래에 제공된 코드는 예상대로 작동하지 않습니다. 
다른 이름으로 배포했다면 모든 코드에서 변수 이름을 수정해야 합니다.

## LLM 체인과 Prompt Engineering에 대한 간단한 소개

체인은 하나 이상의 LLM(Large Language Model)을 논리적인 방식으로 연결함으로써 얻을 수 있는 것으로 Azure OpenAI은 LLM(provider)의 일종입니다. 

체인은 단순(Generic) 또는 특수(Utility)로 나뉠 수 있으며 이번 단계에서는 단순체인(Generic Chain)을 사용할 것입니다. 

* Generic — 단일 LLM은 가장 간단한 체인입니다. 입력 프롬프트와 LLM 이름을 사용한 다음 텍스트 생성(즉, 프롬프트의 출력)을 위해 LLM을 사용합니다.

예제는 다음과 같습니다:

In [9]:
MODEL = "gpt-4-32k" # options: gpt-35-turbo, gpt-35-turbo-16k, gpt-4, gpt-4-32k
COMPLETION_TOKENS = 2000
llm = AzureChatOpenAI(deployment_name=MODEL, temperature=0, max_tokens=COMPLETION_TOKENS)

In [10]:
# 이제 간단한 Prompt Template을 만들어 보겠습니다. 
prompt = PromptTemplate(
    input_variables=["question", "language"],
    template='Answer the following question: "{question}". Give your response in {language}',
)

print(prompt.format(question=QUESTION, language="Korean"))

Answer the following question: "블록체인의 해시함수가 뭐야?". Give your response in Korean


In [11]:
# 그리고 Prompt Template을 사용하여 간단한 Generic 체인을 만들어 보겠습니다. 
chain_chat = LLMChain(llm=llm, prompt=prompt)
chain_chat({"question": QUESTION, "language": "Korean"})

{'question': '블록체인의 해시함수가 뭐야?',
 'language': 'Korean',
 'text': '블록체인의 해시 함수는 데이터를 고정된 길이의 고유한 값으로 변환하는 암호화 방법입니다. 이 함수는 블록체인의 보안성을 높이는 데 중요한 역할을 합니다. 한 번 해시된 데이터는 원래의 형태로 되돌릴 수 없으며, 이는 블록체인의 무결성을 보장합니다. 또한, 해시 함수는 블록체인에서 거래를 검증하고, 새로운 블록을 생성하는 데도 사용됩니다.'}

**노트**: 만약 Resource not found 오류가 발생하면 OpenAI 모델 배포 이름이 위의 변수 MODEL 집합과 다르기 때문일 수 있습니다. 

앞서 본 코드로 인해 이제 간단한 프롬프트를 만들고 ChatGPT 지식을 사용하여 일반적인 질문에 답하는 방법을 알게 되었습니다.

Generic 체인을 독립 실행형 체인으로 사용하는 경우는 거의 없고 보통 Utility 체인의 구성 요소로 사용되는 경우가 더 많습니다. 
또한 주목해야 할 점은 아직 문서나 Azure Search의 결과를 사용하지 않고 있으며, 학습한 데이터에 대한 ChatGPT 지식만 사용하고 있다는 것입니다.

**두 번째 체인 유형은 Utility 체인입니다:**

* Utility — LangChain은 언어 작업을 해결하는데 특화된 여러 LLM으로 구성된 체인입니다. 예를 들어, LangChain은 end-to-end 체인(예: [QA_WITH_SOURCES](https://python.langchain.com/en/latest/modules/chains/index_examples/qa_with_sources.html) for QnA Doc retrieval, Summarization, etc)과 일부 특정 체인 (그래프 생성, 쿼리 및 저장을 위한 GraphQnAChain 등)을 지원합니다. 

이번 워크샵에서는 **qa_with_sources** 라는 특정 체인을 살펴보고 Azure Cognitive Search의 결과를 향상시키는 사용 사례를 시도해 보겠습니다. 

그러나 utility 체인을 이용할때 가장 큰 문제점은 바로 토큰의 크기 입니다. 많은 검색 결과 파일의 내용이 Azure OpenAI에서 제공하는 GPT 모델의 허용 토큰보다 클 수 있습니다.

이것이 해결하기 위해 나온 것이 바로 임베딩/벡터의 개념입니다. 

## 임베딩과 벡터 검색 

임베딩은 기계 학습 모델과 알고리즘에 의해 쉽게 활용될 수 있는 데이터 표현의 특별한 형식입니다. 임베딩은 텍스트의 semantic 정보를 밀도 있게 표현하는 것으로 각 임베딩은 부동 소수점 숫자의 벡터이므로 벡터 공간에서 두 임베딩 사이의 거리가 원래 형식의 두 입력 사이의 의미적 유사성과 상관관계가 있습니다. 예를 들어, 두 텍스트의 의마가 유사한 경우 벡터 표현도 유사해야 합니다.

LLM(Language Model)의 토큰 제한 문제를 해결하기 위해 솔루션은 다음 단계를 포함합니다:

1. **문서 세분화**: 문서를 더 작은 세그먼트 또는 청크로 나눕니다.
2. **청크 벡터화**: 적절한 기술을 사용하여 이러한 청크를 벡터로 변환합니다.
3. **벡터 시맨틱 검색**: 주어진 질문과 유사한 상위 청크를 식별하기 위해 벡터를 사용하여 시맨틱 검색을 실행합니다.
4. **최적의 컨텍스트 제공**: LLM에 가장 관련성 있고 간결한 컨텍스트를 제공하여 포괄성과 길이 간의 최적의 균형을 달성합니다.


이번 notebook에서의 목표는 벡터 인덱스를 사용하는 것입니다. 다양한 파일 형식에 대해 OCR로 파서를 수동으로 코드화하고 인덱스와 데이터를 동기화할 수 있는 스케줄러를 개발하는 것이 가능하지만, 더 효율적인 대안이 있습니다. 

1. 청크와 벡터를 벡터 기반 인덱스로 수동으로 푸시합니다.
2. 사용자가 필요한 문서를 검색할 때 벡터 기반 인덱스 작성합니다. 
3. 사용자 지정 기술(청킹 및 벡터화용)을 사용하고 지식 저장소를 사용하여 수집 시점에 텍스트 기반 ai가 풍부한 인덱스에서 벡터 기반 인덱스를 만듭니다. 
이 작업을 수행하는 방법은 [여기](https://github.com/Azure/cognitive-search-vector-pr/blob/main/demo-python/code/azure-search-vector-ingestion-python-sample.ipynb)에서 확인 가능합니다. 

이 notebook에서는 옵션 2를 구현할 예정입니다. **각 텍스트 기반 인덱스별로 벡터 기반 인덱스를 생성하고 문서가 검색될 때마다 필요에 따라 인덱스를 채우십시오**. 

노트 1과 노트 2에서 볼 수 있듯이 각 텍스트 기반 인덱스에는 아직 사용하지 않은 vector화된 필드가 포함되어 있습니다. 이제 이 필드를 활용할 것 입니다. 
이번 단계의 목적은 수집 시점에 모든 문서를 벡터화하는 것을 피하며(옵션 3) 사용자가 문서를 검색할 때만 문서 청크를 벡터화합니다. 이 접근 방식은 문서가 실제로 필요할 때만 자금과 자원을 할당하기에 일반적으로 Data Lake에 있는 문서 중 20%만 자주 액세스되고 나머지는 손상되지 않은 상태로 유지됩니다. 이 방법론을 [파레토 원칙](https://en.wikipedia.org/wiki/Pareto_principle) 에서 가져왔습니다. 

In [12]:
index_name = "cogsrch-index-files"
index2_name = "cogsrch-index-csv"
indexes = [index_name, index2_name]

코드에서 자주 쓰이는 함수들을 반복하지 않기 위해 common/utils.py 파일과 common/prompts.py 파일 넣어두고 필요할 때 불러 사용하고 있습니다. 

In [13]:
k = 10 # Number of results per each text_index
ordered_results = get_search_results(QUESTION, indexes, k=10, reranker_threshold=1)
print("Number of results:",len(ordered_results))

Number of results: 1


In [14]:
# Uncomment the below line if you want to inspect the ordered results
# ordered_results

이제 사용자가 텍스트 기반 인덱스를 사용하여 문서를 검색할 때 벡터 기반 인덱스를 사용할 수 있습니다. 이 접근 방식은 사용자 쿼리당 두 번의 검색(텍스트 기반 인덱스와 벡터 기반 인덱스)이 필요하지만 구현이 더 간단하며 사용자가 시스템을 사용할 때 점점 더 빨라질 것 입니다.

In [15]:
embedder = OpenAIEmbeddings(deployment="text-embedding-ada-002", chunk_size=1) 

In [16]:
%%time
for key,value in ordered_results.items():
    if value["vectorized"] != True: # 문서가 아직 벡터링되지 않은 경우
        i = 0
        print("Vectorizing",len(value["chunks"]),"chunks from Document:",value["location"])
        for chunk in value["chunks"]: # 문서의 텍스트 청크를 반복
            try:
                upload_payload = {  # 벡터 기반 인덱스에 청크와 벡터 삽입
                    "value": [
                        {
                            "id": key + "_" + str(i),
                            "title": f"{value['title']}_chunk_{str(i)}",
                            "chunk": chunk,
                            "chunkVector": embedder.embed_query(chunk if chunk!="" else "-------"),
                            "name": value["name"],
                            "location": value["location"],
                            "@search.action": "upload"
                        },
                    ]
                }

                r = requests.post(os.environ['AZURE_SEARCH_ENDPOINT'] + "/indexes/" + value["index"]+"-vector" + "/docs/index",
                                     data=json.dumps(upload_payload), headers=headers, params=params)
                
                if r.status_code != 200:
                    print(r.status_code)
                    print(r.text)
                else:
                    i = i + 1 # 청크의 수 증가
                    
                    # 문서 안 텍스트 기반 인덱스 업데이트하고 "벡터화"로 표시
                    upload_payload = {
                        "value": [
                            {
                                "id": key,
                                "vectorized": True,
                                "@search.action": "merge"
                            },
                        ]
                    }

                    r = requests.post(os.environ['AZURE_SEARCH_ENDPOINT'] + "/indexes/" + value["index"]+ "/docs/index",
                                     data=json.dumps(upload_payload), headers=headers, params=params)
                    
                    
            except Exception as e:
                print("Exception:",e)
                print(content)
                continue

Vectorizing 4 chunks from Document: https://holstorage.blob.core.windows.net/arxivcs/blockchain_explaination.pdf
CPU times: total: 1.58 s
Wall time: 13.5 s


**노트**: 텍스트 기반 인덱스와 벡터 기반 인덱스 동기화
문서 변경의 경우 파일에 새 버전이 있으면 Azure Engine이 텍스트 기반 인덱스를 자동으로 업데이트합니다. 이렇게 하면 다음 번에 파일이 검색되면 새로운 벡터 기반 인덱스로 다시 벡터가 덮어씌이게 됩니다.
그러나 파일 삭제의 경우 Azure Search 엔진은 원본에서 파일이 삭제되면 텍스트 기반 인덱스의 문서를 삭제하지만 텍스트 기반 인덱스에서 삭제된 ID를 찾고 벡터 기반 인덱스에서 해당 청크를 삭제하는 것은 고정된 일정에 따라 실행되는 스크립트를 코딩해야 합니다.

이제 벡터 기반 인덱스를 검색하여 질문과 가장 유사한 상위 k개의 청크를 얻습니다:

In [17]:
vector_indexes = [index+"-vector" for index in indexes]

k = 10
similarity_k = 3
ordered_results = get_search_results(QUESTION, vector_indexes,
                                        k=k, # Number of results per vector index
                                        reranker_threshold=1,
                                        vector_search=True, 
                                        similarity_k=similarity_k,
                                        query_vector = embedder.embed_query(QUESTION)
                                        )
print("Number of results:",len(ordered_results))

Number of results: 3


벡터 검색의 경우 LLM에 컨텍스트로 k=5개 이상의 청크(각각 최대 5000자)를 제공하지 않는 것이 좋습니다. 그렇지 않으면 나중에 토큰 제한으로 메모리와 대화를 시도하는 문제가 발생할 수 있습니다.

In [18]:
top_docs = []
for key,value in ordered_results.items():
    location = value["location"] if value["location"] is not None else ""
    top_docs.append(Document(page_content=value["chunk"], metadata={"source": location}))
        
print("Number of chunks:",len(top_docs))

Number of chunks: 3


In [19]:
# 문서의 토큰 수 계산
if(len(top_docs)>0):
    tokens_limit = model_tokens_limit(MODEL) # utils.py에 있는 사용자 정의 함수
    prompt_tokens = num_tokens_from_string(COMBINE_PROMPT_TEMPLATE) # utils.py에 있는 사용자 정의 함수
    context_tokens = num_tokens_from_docs(top_docs) # utils.py에 있는 사용자 정의 함수
    
    requested_tokens = prompt_tokens + context_tokens + COMPLETION_TOKENS
    
    chain_type = "map_reduce" if requested_tokens > 0.9 * tokens_limit else "stuff"  
    
    print("System prompt token count:",prompt_tokens)
    print("Max Completion Token count:", COMPLETION_TOKENS)
    print("Combined docs (context) token count:",context_tokens)
    print("--------")
    print("Requested token count:",requested_tokens)
    print("Token limit for", MODEL, ":", tokens_limit)
    print("Chain Type selected:", chain_type)
        
else:
    print("NO RESULTS FROM AZURE SEARCH")

System prompt token count: 1669
Max Completion Token count: 2000
Combined docs (context) token count: 8782
--------
Requested token count: 12451
Token limit for gpt-4-32k : 32768
Chain Type selected: stuff


이제 LangChain 'qa_with_sources'의 유틸리티 체인을 사용하겠습니다.

In [20]:
if chain_type == "stuff":
    chain = load_qa_with_sources_chain(llm, chain_type=chain_type, 
                                       prompt=COMBINE_PROMPT)
elif chain_type == "map_reduce":
    chain = load_qa_with_sources_chain(llm, chain_type=chain_type, 
                                       question_prompt=COMBINE_QUESTION_PROMPT,
                                       combine_prompt=COMBINE_PROMPT,
                                       return_intermediate_steps=True)

In [21]:
%%time
# 다른 언어로도 가능합니다. 
response = chain({"input_documents": top_docs, "question": QUESTION, "language": "Korean"})

CPU times: total: 0 ns
Wall time: 16.9 s


In [22]:
display(Markdown(response['output_text']))

블록체인의 해시함수는 데이터를 고정된 길이의 데이터로 변환하는 것을 의미하며, 이는 '단방향 변환'이기 때문에 해시값을 이용해서 원본 데이터를 복원할 수 없습니다<sup><a href="https://holstorage.blob.core.windows.net/arxivcs/blockchain_explaination.pdf" target="_blank">[1]</a></sup>. 블록체인에서는 이 해시값을 이용해 해당 블록에 서명하고 이전 블록의 해시값을 다음 블록에 기록함으로써 체인 형태의 연결 리스트를 형성하게 됩니다<sup><a href="https://holstorage.blob.core.windows.net/arxivcs/blockchain_explaination.pdf" target="_blank">[1]</a></sup>. 또한, 블록체인에서 사용되는 암호기술은 해시함수, 전자서명, 공개키 암호화 알고리즘이 있습니다<sup><a href="https://holstorage.blob.core.windows.net/arxivcs/blockchain_explaination.pdf" target="_blank">[1]</a></sup>.

**참고**: 답변의 높은 정확도와 품질에도 불구하고 COMBINE_PROMPT에 제시된 지침대로 참조가 이루어지지 않는 경우가 있습니다. 이러한 동작은 GPT-3.5 모델을 사용할 때 나타나는데, 이 문제는 노트 5의 말미에 자세히 알아보겠습니다. 

In [23]:
# Uncomment if you want to inspect the results from map_reduce chain type, each top similar chunk summary (k=4 by default)

# if chain_type == "map_reduce":
#     for step in response['intermediate_steps']:
#         display(HTML("<b>Chunk Summary:</b> " + step))

# Summary
##### Azure Cognitive Search의 결과 도출의 대한 요약은 다음과 같습니다:
- Azure Cognitive Search를 활용하여 각 인덱스에서 상위 문서를 식별하는 다중 인덱스 텍스트 기반 검색을 수행합니다.
- Azure Cognitive Search의 벡터 검색을 이용하여 가장 관련성이 높은 정보 덩어리를 추출합니다.
- 그 다음, Azure OpenAI는 추출된 청크를 컨텍스트로 활용하고 내용을 이해한 후 이를 활용하여 최적의 답변을 제공합니다.